# 1 in 1 out numeric test

A single numeric value in and a single numeric value out.

## Build and save the graph

Ensure you are working in a virtual environment with tensorflow >= 1.9 installed, and in the notebook's directory on the command line:

```bash
rm -r saved_graph
```

In [2]:
import tensorflow as tf

In [19]:
# this graph takes int32 inputs and produces int32 outputs (via tf.cast)
# but its internal operations are still floating point ops

tf.reset_default_graph()

x_int32 = tf.placeholder(tf.int32, shape=(1), name="input")
w = tf.Variable(10, dtype=tf.float32, name="w")
b = tf.Variable(5, dtype=tf.float32, name="b")

x_float = tf.cast(x_int32, dtype=tf.float32)
z_float = tf.add(tf.multiply(x_float, w), b)
z_int32 = tf.cast(z_float, dtype=tf.int32, name="output")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    z_eval = sess.run([z_int32], feed_dict={x_int32: [2]})
    
    tf.saved_model.simple_save(sess, "./saved_graph", inputs={"input": x_int32}, outputs={"output": z_int32})
    
    print(z_eval) # 25

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_graph/saved_model.pb
[array([25], dtype=int32)]


## Freeze the graph (Unused)

Having used the `tf.saved_model.simple_save` function, now use the `freeze_graph` cli. Tensorflow also provides a python interface to the freeze_graph function.

On the command line:

```bash
mkdir frozen_graph

freeze_graph \
    --input_saved_model_dir=saved_graph \
    --saved_model_tags=serve \
    --output_graph=frozen_graph/frozen_graph.pb \
    --output_node_names=output
```

## Convert the model to a tensorflow lite format (Unused)

Use the `toco` command line utility to convert the frozen graph to a tensorflow lite compatible file:

```bash
mkdir tflite

toco \
	--graph_def_file=frozen_graph/frozen_graph.pb \
	--output_file=tflite/model.tflite \
	--input_arrays=input \
	--input_shapes=1 \
	--output_arrays=output
```

# Adding Quantization (Unused)

In [ ]:
tf.reset_default_graph()



## Quantize the model in a tflite format (Unused)

Use the `toco` command line utility to convert the frozen graph to a quantized tensorflow lite compatible file:

```bash
mkdir tflite

toco \
	--graph_def_file=frozen_graph/frozen_graph.pb \
	--output_file=tflite/model.tflite \
	--input_arrays=input_x \
	--input_shapes=1 \
	--output_arrays=output_z \
    --inference_type=QUANTIZED_UINT8 \
    --std_dev_values=127 \
    --mean_values=12 
```